In [26]:
variables = {
 
    'y_1': {'b_1': 1.0,
          'constraint_coeffs':
            {'parking_fee': 3.5,
           'expected_avail': 2,
           'et_next_out': 70.0,
           'et_next_in': 27.0,
           'walk_time': 5.0,
           'travel_et': 22.0}},
    
    'y_2': {'b_2': 1.0,
          'constraint_coeffs':
            {'parking_fee': 4.5,
           'expected_avail': 2,
           'et_next_out': 8.0,
           'et_next_in': 27.0,
           'walk_time': 7.0,
           'travel_et': 23.1}},
    
    'y_3': {'b_3': 1.0,
          'constraint_coeffs':
            {'parking_fee': 5.5,
           'expected_avail': 2,
           'et_next_out': 9.0,
           'et_next_in': 27.0,
           'walk_time': 8.233,
           'travel_et': 24.0}},
    
#     'y_4': {'b_4': 1.0,
#           'constraint_coeffs':
#             {'parking_fee': 5.5,
#            'expected_avail': 2,
#            'et_next_out': 15.0,
#            'et_next_in': 27.0,
#            'walk_time': 15.233,
#            'travel_et': 20.11}},
    
#     'y_5': {'b_5': 1.0,
#           'constraint_coeffs':
#             {'parking_fee': 7.5,
#            'expected_avail': 2,
#            'et_next_out': 15.0,
#            'et_next_in': 27.0,
#            'walk_time': 5.0,
#            'travel_et': 20.5}}
}

'''
User specs
'''
stay_duration = 1.75 * 60 #In minutes
p_budget = 8.00 #dollars

In [27]:
import pulp
from pulp import *
import numpy as np
import pandas as pd

model = LpProblem("ParkingOptimizer", LpMinimize)

#Decision variables
var_idx = list(range(1, len(variables)+1))
lp_vars = LpVariable.dicts('y', var_idx, lowBound=0, upBound=1)
print(lp_vars)

#Objective function!
model += lpSum([variables[str(lp_vars[vi])][f'b_{vi}'] * lp_vars[vi] for vi in var_idx])


#Availability
model += lpSum([variables[str(lp_vars[v_i])]['constraint_coeffs'][f'expected_avail'] * lp_vars[v_i]
                for v_i in var_idx]) >= np.mean([variables[str(lp_vars[v_i])]['constraint_coeffs'][f'expected_avail']for v_i in var_idx])
    
#Rates
model += lpSum([-1 * variables[str(lp_vars[v_i])]['constraint_coeffs'][f'parking_fee'] * lp_vars[v_i] for v_i in var_idx]) >= -p_budget

model += lpSum([-1 * variables[str(lp_vars[v_i])]['constraint_coeffs'][f'travel_et'] * lp_vars[v_i] for v_i in var_idx]) >= -np.median([variables[str(lp_vars[v_i])]['constraint_coeffs'][f'travel_et']for v_i in var_idx])
model += lpSum([-1 * variables[str(lp_vars[v_i])]['constraint_coeffs'][f'walk_time'] * lp_vars[v_i] for v_i in var_idx]) >= -np.mean([variables[str(lp_vars[v_i])]['constraint_coeffs'][f'walk_time']for v_i in var_idx])

model += lpSum([-1 * variables[str(lp_vars[v_i])]['constraint_coeffs'][f'et_next_out'] * lp_vars[v_i] for v_i in var_idx]) >= -np.mean([variables[str(lp_vars[v_i])]['constraint_coeffs'][f'et_next_out']for v_i in var_idx])

#Probabilistic
# model += lpSum([1 * lp_vars[vi] for vi in var_idx]) == 1

model

{1: y_1, 2: y_2, 3: y_3}


ParkingOptimizer:
MINIMIZE
1.0*y_1 + 1.0*y_2 + 1.0*y_3 + 0.0
SUBJECT TO
_C1: 2 y_1 + 2 y_2 + 2 y_3 >= 2

_C2: - 3.5 y_1 - 4.5 y_2 - 5.5 y_3 >= -8

_C3: - 22 y_1 - 23.1 y_2 - 24 y_3 >= -23.1

_C4: - 5 y_1 - 7 y_2 - 8.233 y_3 >= -6.74433333333

_C5: - 7 y_1 - 8 y_2 - 9 y_3 >= -8

VARIABLES
y_1 <= 1 Continuous
y_2 <= 1 Continuous
y_3 <= 1 Continuous

In [28]:
#SOLVE
model.solve()

print("*"*30 + "STATUS" + "*"*30)
print(f'Solution status: {LpStatus[model.status]}')
print("~~~ Values ~~~")
for variable in model.variables():
        print(variable.name, " =", variable.varValue)
# print(f"Maximum profit (under constraints) = ${pulp.value(lp_problem.objective)}")
print("*"*30 + "***" + "*"*30)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/ahnafayub/opt/anaconda3/envs/geodat/lib/python3.10/site-packages/pulp/apis/../solverdir/cbc/osx/64/cbc /var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/b988514590244a1a9152050eef95da6b-pulp.mps timeMode elapsed branch printingOptions all solution /var/folders/52/md6yz3dn2g344wf8pqqb7kc80000gn/T/b988514590244a1a9152050eef95da6b-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 10 COLUMNS
At line 29 RHS
At line 35 BOUNDS
At line 39 ENDATA
Problem MODEL has 5 rows, 3 columns and 15 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Presolve 5 (0) rows, 3 (0) columns and 15 (0) elements
0  Obj 0 Primal inf 0.9999999 (1)
1  Obj 1
Optimal - objective value 1
Optimal objective 1 - 1 iterations time 0.002
Option for printingOptions changed from normal to all
Total time (CPU seconds):       0.00   (Wallclock seconds):       0.

In [29]:
optimal_weights = {}
for variable in model.variables():
    optimal_weights[str(variable.name)] = variable.varValue
    # if variable.varValue > 0:
    #     print(variables[variable.name])
    #     nonzero_vars.append(str(variable))
        
variables
cols = ['var','optimal_weight' ,'parking_fee', 'expected_avail','travel_et', 'walk_time', 'et_next_out', 'et_next_in']
res = []
for i in var_idx:
    res.append([f'y_{i}',optimal_weights[f'y_{i}'] , variables[f'y_{i}']['constraint_coeffs']['parking_fee'],
                                    variables[f'y_{i}']['constraint_coeffs']['expected_avail'],
                variables[f'y_{i}']['constraint_coeffs']['travel_et'],
                variables[f'y_{i}']['constraint_coeffs']['walk_time'],
                                     variables[f'y_{i}']['constraint_coeffs']['et_next_out'],
                                       variables[f'y_{i}']['constraint_coeffs']['et_next_in']
               ])
res = pd.DataFrame(res, columns=cols)
res.sort_values(by=['optimal_weight','travel_et', 'walk_time'],
                ascending = [False, True, True], inplace=True)
res

,var,optimal_weight,parking_fee,expected_avail,travel_et,walk_time,et_next_out,et_next_in
0,y_1,1.0,3.5,2,22.0,5.000,7.0,27.0
1,y_2,0.0,4.5,2,23.1,7.000,8.0,27.0
2,y_3,0.0,5.5,2,24.0,8.233,9.0,27.0


In [30]:
res.set_index('var').loc[nonzero_vars]

,optimal_weight,parking_fee,expected_avail,travel_et,walk_time,et_next_out,et_next_in
var,,,,,,,
y_2,0.0,4.5,2,23.1,7.0,8.0,27.0
